In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### 증강 전 초기 데이터셋 : cross, v_marker, tray

from glob import glob
input_dir = "/content/drive/MyDrive/yolov5n/marker_augmentation/images"

# 이미지 확장자별로 확인
jpeg_files = glob(f"{input_dir}/*.jpeg")
jpg_files = glob(f"{input_dir}/*.jpg")
png_files = glob(f"{input_dir}/*.png")

print(f"jpeg: {len(jpeg_files)}")
print(f"jpg: {len(jpg_files)}")
print(f"png: {len(png_files)}")

jpeg: 5
jpg: 0
png: 0


In [ ]:
### 데이터셋 경로 지정해둘 marker.yaml 생성, 파인튜닝 시에 수정

yaml_content = """
path: /content/drive/MyDrive/yolo_backup/marker_augmentation/dataset
train: images/train
val: images/val
names:
  - cross_marker
  - v_marker
"""

# YAML 파일 저장 경로
yaml_path = "/content/drive/MyDrive/yolov5n/marker_augmentation/marker.yaml"

# 파일로 저장
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("✅ marker.yaml 파일이 저장되었습니다.")

✅ marker.yaml 파일이 저장되었습니다.


In [ ]:
'''
초기 데이터셋인 cross, v_marker 데이터 증강.
추가로 aruco 마커도 별도 클래스로 나눠봤지만 v_marker와의 오류가 다수 생기므로
aruco 마커는 opencv에서만 활용하는 편이 나아 2개의 클래스만 우선 증강함.
yolov5n 기준 클래스 별 300장 정도로 데이터셋 구성
'''

import os
import cv2
import albumentations as A
from glob import glob
from tqdm import tqdm

# 새 입력/출력 경로
input_dir = "/content/drive/MyDrive/yolov5n/marker_augmentation/images"
output_img_dir = "/content/drive/MyDrive/yolov5n/marker_augmentation/dataset/images/train"
output_lbl_dir = "/content/drive/MyDrive/yolov5n/marker_augmentation/dataset/labels/train"

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)

# 증강 파이프라인 (드론 시점 최적화)
transform = A.Compose([
    A.RandomResizedCrop(size=(640, 640), scale=(0.3, 1.0), ratio=(0.75, 1.33), p=0.5),
    A.Perspective(scale=(0.05, 0.2), p=0.4),
    A.HorizontalFlip(p=0.4),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.5),
    A.MotionBlur(blur_limit=5, p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.3),
    A.Affine(scale=(0.8, 1.2), rotate=(-25, 25), shear=(-10, 10), translate_percent=0.1, p=0.6),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# 원본 이미지 목록 불러오기
image_paths = glob(os.path.join(input_dir, "*.jpeg"))

# 클래스당 증강 수 설정
NUM_AUGS_PER_CLASS = 300

for img_path in tqdm(image_paths):
    fname = os.path.splitext(os.path.basename(img_path))[0]
    img = cv2.imread(img_path)
    if img is None:
        print(f"이미지 로드 실패: {img_path}")
        continue

    class_label = 0 if "cross" in fname else 1
    dummy_box = [0.5, 0.5, 1.0, 1.0]

    for i in range(NUM_AUGS_PER_CLASS):
        try:
            transformed = transform(image=img, bboxes=[dummy_box], class_labels=[class_label])
            aug_img = transformed['image']
            aug_box = transformed['bboxes'][0]

            out_img_path = os.path.join(output_img_dir, f"{fname}_aug_{i}.jpg")
            out_lbl_path = os.path.join(output_lbl_dir, f"{fname}_aug_{i}.txt")

            cv2.imwrite(out_img_path, aug_img)
            with open(out_lbl_path, "w") as f:
                f.write(f"{class_label} {' '.join([str(round(x, 6)) for x in aug_box])}\n")

        except Exception as e:
            print(f"[오류 발생: {fname}_aug_{i}] → {e}")

/tmp/ipython-input-19-4124029528.py:23: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
100%|██████████| 2/2 [02:39<00:00, 79.61s/it]


In [ ]:
### yolov5 클론

%cd /content
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

In [ ]:
### 추가된 tray 클래스 train,val,test 분할코드

import os
import random
import shutil

# 경로 설정
base_path = "/content/marker_augmentation/dataset/images/tray_augmented"
image_files = [f for f in os.listdir(base_path) if f.endswith(".jpg")]

# 셔플 후 split
random.seed(42)
random.shuffle(image_files)

train_split = int(len(image_files) * 0.7)
val_split = int(len(image_files) * 0.9)  # 70% + 20% = 90%

train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

# 폴더 준비
target_base = "/content/marker_augmentation/dataset/images"
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(target_base, split), exist_ok=True)

# 복사 함수
def move_images(files, split):
    for file in files:
        src = os.path.join(base_path, file)
        dst = os.path.join(target_base, split, file)
        shutil.copy(src, dst)

# 실행
move_images(train_files, "train")
move_images(val_files, "val")
move_images(test_files, "test")

print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

Train: 210, Val: 60, Test: 30


In [ ]:
### tray 클래스(인덱스 2) 데이터셋 분할 확인 코드

import os
from collections import Counter

base = "/content/marker_augmentation/dataset"
splits = ['train', 'val', 'test']

for split in splits:
    label_dir = os.path.join(base, f"labels/{split}")
    class_counts = Counter()

    if not os.path.exists(label_dir):
        print(f"[{split.upper()}] 라벨 폴더가 존재하지 않습니다.")
        continue

    for file in os.listdir(label_dir):
        if file.endswith('.txt'):
            with open(os.path.join(label_dir, file), 'r') as f:
                for line in f:
                    cls_index = int(line.strip().split()[0])
                    class_counts[cls_index] += 1

    print(f"[{split.upper()}] 클래스 인덱스 분포: {dict(class_counts)}")

[TRAIN] 클래스 인덱스 분포: {2: 210}
[VAL] 클래스 인덱스 분포: {2: 60}
[TEST] 클래스 인덱스 분포: {2: 30}


In [ ]:
### 학습에 사용될 dataset 크기 확인 코드

import os
from glob import glob

# 경로 지정 (증강된 train 디렉토리)
img_dir = "/content/drive/MyDrive/yolov5n/marker_augmentation/dataset/images/train"
lbl_dir = "/content/drive/MyDrive/yolov5n/marker_augmentation/dataset/labels/train"

# 파일 목록 불러오기
img_paths = glob(os.path.join(img_dir, "*.jpg"))
lbl_paths = glob(os.path.join(lbl_dir, "*.txt"))

# 클래스별 개수 추출
cross_imgs = [f for f in img_paths if "cross" in os.path.basename(f)]
v_imgs = [f for f in img_paths if "v_marker" in os.path.basename(f)]

# 출력
print(f"✅ 총 이미지 수: {len(img_paths)}")
print(f"    └─ cross_marker: {len(cross_imgs)}장")
print(f"    └─ v_marker: {len(v_imgs)}장\n")
print(f"✅ 총 라벨 파일 수: {len(lbl_paths)}")

✅ 총 이미지 수: 480
    └─ cross_marker: 240장
    └─ v_marker: 240장

✅ 총 라벨 파일 수: 480


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
### 증강 이미지의 라벨링(.txt) 복사/정렬 코드

import os
import shutil

# 원본 라벨이 있는 경로
original_labels = {
    "tray1": "/content/drive/MyDrive/yolov5n/marker_augmentation/tray_labels/tray1.txt",
    "tray2": "/content/drive/MyDrive/yolov5n/marker_augmentation/tray_labels/tray2.txt",
    "tray3": "/content/drive/MyDrive/yolov5n/marker_augmentation/tray_labels/tray3.txt"
}

# 증강 이미지가 있는 루트
base = "/content/marker_augmentation/dataset"

for split in ['train', 'val', 'test']:
    img_dir = os.path.join(base, f'images/{split}')
    lbl_dir = os.path.join(base, f'labels/{split}')
    os.makedirs(lbl_dir, exist_ok=True)

    for img_file in os.listdir(img_dir):
        if img_file.endswith(".jpg"):
            base_name = img_file.split("_aug")[0]
            label_path = original_labels.get(base_name)
            if label_path:
                label_file = os.path.join(lbl_dir, img_file.replace(".jpg", ".txt"))
                shutil.copy(label_path, label_file)

In [ ]:
### tray 클래스를 추가하여 학습

!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/drive/MyDrive/yolov5n/marker_augmentation/marker.yaml \
  --weights /content/drive/MyDrive/yolov5n/train_logs/marker_yolov5n/weights/best.pt \
  --name marker_vx_tray_finetuned \
  --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-04 17:13:21.604883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751649201.626651   27946 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751649201.633104   27946 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=/content/drive/MyDrive/yolov5n/train_logs/marker_yolov5n/weight

In [ ]:
### 완성된 모델에서의 클래스(인덱스) 확인

from ultralytics import YOLO

model = YOLO("runs/train/marker_vx_tray_finetuned/weights/best.pt")
print("✅ 클래스 수:", model.model.names)

✅ 클래스 수: {0: 'cross_marker', 1: 'v_marker', 2: 'tray'}


In [ ]:
### test 데이터를 사용해 테스트 - marker.yaml 파일의 경로 수정

!python val.py \
  --img 640 \
  --batch 16 \
  --data /content/drive/MyDrive/yolov5n/marker_augmentation/marker.yaml \
  --weights runs/train/marker_vx_tray_finetuned/weights/best.pt \
  --task test

val: data=/content/drive/MyDrive/yolov5n/marker_augmentation/marker.yaml, weights=['runs/train/marker_vx_tray_finetuned/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
test: Scanning /content/marker_augmentation/dataset/labels/test... 30 images, 0 backgrounds, 0 corrupt: 100% 30/30 [00:00<00:00, 2133.93it/s]
test: New cache created: /content/marker_augmentation/dataset/labels/test.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 2/2 [00:01<00:00,  1.69it/s]
                   all         30    

In [ ]:
### 완성된 모델 파일 best.pt을 onxx로 변환

%cd /content/yolov5/yolov5

!python export.py \
  --weights runs/train/marker_vx_tray_finetuned/weights/best.pt \
  --include onnx \
  --img 640 \
  --device cpu

/content/yolov5/yolov5
export: data=data/coco128.yaml, weights=['runs/train/marker_vx_tray_finetuned/weights/best.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs

PyTorch: starting from runs/train/marker_vx_tray_finetuned/weights/best.pt with output shape (1, 25200, 8) (3.7 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 2.0s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.18.0...
ONNX: export su